In [1]:
import glob
import os
import numpy as np
from ip_basic.depth_map_utils import *
from ip_basic.vis_utils import *
from PIL import Image
import time

from concurrent.futures import ThreadPoolExecutor
import threading
import multiprocessing

import pickle as pkl


## Multithreading settings for saving pickle files
MULTITHREADED_PROCESSING = False
NUM_THREADS = 8


In [2]:
MAX_DEPTH = 10.00
DEPTH_PARAM1 = 351.30
DEPTH_PARAM2 = 1092.50

In [3]:
# Best settings so far: 
fill_type = 'fast'
extrapolate = True
blur_type = None

# Fast fill kernels
custom_kernel = DIAMOND_KERNEL_7
morph_kernel = FULL_KERNEL_5
dilation_kernel = FULL_KERNEL_7

In [4]:
# Mutex and lock and global counter
mutex = threading.Lock()
pickle_counter = 0

In [5]:
def convert_to_metres(depth_image):
    return DEPTH_PARAM1 / (DEPTH_PARAM2 - depth_image)

def read_pgm(filename):
    """Return a raster of integers from a PGM as a list of lists."""
    raster = []
    try:
        with open(filename, 'rb') as pgmf:
            header = pgmf.readline()
            assert header[:2] == b'P5'
            (width, height) = [int(i) for i in header.split()[1:3]]
            depth = int(header.split()[3])
            assert depth <= 65535
            for y in range(height):
                row = []
                for y in range(width):
                    low_bits = ord(pgmf.read(1))
                    row.append(low_bits+255*ord(pgmf.read(1)))
                raster.append(row)
    except:
        print("Error reading file: {}".format(filename))
        return None
    return np.array(raster)

In [6]:
class SingleImageProcessSequence:
    def __init__(self, projected_depths, final_depths, filled_depths):
        self.projected_depths = projected_depths
        self.final_depths = final_depths
        self.filled_depths = filled_depths

# # Code to fill unknown pixels with the nearest known pixel
# def fill_empty_spaces(depth_image):
#     .... something here


def process_images_multithreaded(depth_image_list):
    # Declare global variables from above snippet
    global MAX_DEPTH
    global DEPTH_PARAM1
    global DEPTH_PARAM2
    global fill_type
    global extrapolate
    global blur_type
    global custom_kernel
    global morph_kernel
    global dilation_kernel
    global pickle_counter

    image_array = []
    image_num = 0
    for depth_path in depth_image_list:
        print(image_num)
        try:
            # Read depth image
            depth_image = read_pgm(depth_path)

            # Convert to metres
            projected_depths = convert_to_metres(depth_image)
            projected_depths = np.clip(projected_depths, 0, MAX_DEPTH - 0.03)
            projected_depths = (projected_depths).astype(np.float32)

            # Fill in
            if fill_type == 'fast':
                completed_depths = fill_in_fast(
                    projected_depths, max_depth=MAX_DEPTH, extrapolate=extrapolate, blur_type=blur_type, 
                    morph_kernel=morph_kernel, dilation_kernel=dilation_kernel)
                filled_depths = fill_empty_spaces(completed_depths)
                image_array.append(SingleImageProcessSequence(projected_depths, completed_depths, filled_depths))
                # image_array.append(final_depths)
            else:
                raise ValueError('Invalid fill_type {}'.format(fill_type))
        except:
            print("Error processing file: {}".format(depth_path))
            continue
    
    # save image array as a pickle
    mutex.acquire()
    pickle_counter += 1
    pickle_filename = 'pickle_' + str(pickle_counter) + '.pkl'
    mutex.release()

    with open(pickle_filename, 'wb') as f:
        pkl.dump(image_array, f)
        print("Dumped one pickle")


SyntaxError: invalid syntax (1139303687.py, line 9)

In [7]:
# List all file paths of all files with .pgm extension in a list recursively
# depth_image_paths = glob.glob('/media/mihir/MKSSD/misc_offices_playroom_reception_studies_study_rooms/**/*.pgm', recursive=True)
# depth_image_paths = glob.glob('/media/mihir/MKSSD/cafe_dining_furniture_home_off_kitchen_libraries_living_rooms/**/*.pgm', recursive=True)
depth_image_paths = glob.glob('/media/mihir/MKSSD/basements_bedrooms_bathrooms_bookstores/**/*.pgm', recursive=True)

tenth_of_depth_images = depth_image_paths[::10]

save_to_file(tenth_of_depth_images)



size_per_pickle = 1000
list_of_lists = [tenth_of_depth_images[i:i + size_per_pickle] for i in range(0, len(tenth_of_depth_images), size_per_pickle)]

print("Number of lists: {}".format(len(list_of_lists)))

print('Number of depth images: {}'.format(len(depth_image_paths)))
print('Number of depth images to process: {}'.format(len(tenth_of_depth_images)))

Number of lists: 16
Number of depth images: 158662
Number of depth images to process: 15867


In [8]:
# Run multiple processes to call process_images_multithreaded on each depth image
if MULTITHREADED_PROCESSING:
    with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
        for result in executor.map(process_images_multithreaded, list_of_lists):
            print("Finished thread")
else:
    for i in range(len(list_of_lists)):
        print("Global counter: {}".format(pickle_counter))
        print("Processing list {}".format(i))
        process_images_multithreaded(list_of_lists[i])


Global counter: 0
Processing list 0
Global counter: 1
Processing list 1
Global counter: 2
Processing list 2
Global counter: 3
Processing list 3
Error reading file: /media/mihir/MKSSD/basements_bedrooms_bathrooms_bookstores/bedrooms_part1/bedroom_0002/d-1294599020.394699-1677996343.pgm
Error processing file: /media/mihir/MKSSD/basements_bedrooms_bathrooms_bookstores/bedrooms_part1/bedroom_0002/d-1294599020.394699-1677996343.pgm
Global counter: 4
Processing list 4
Global counter: 5
Processing list 5
Global counter: 6
Processing list 6
Global counter: 7
Processing list 7
Global counter: 8
Processing list 8
Global counter: 9
Processing list 9
Global counter: 10
Processing list 10
Global counter: 11
Processing list 11
Error reading file: /media/mihir/MKSSD/basements_bedrooms_bathrooms_bookstores/bedrooms_part6/bedroom_0111/d-1315239083.116235-3655284704.pgm
Error processing file: /media/mihir/MKSSD/basements_bedrooms_bathrooms_bookstores/bedrooms_part6/bedroom_0111/d-1315239083.116235-36552